# Train our Philosophical Document Embedding Model

In [1]:
import torch

import loader
import models
import utility


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

c:\Users\lucas\Desktop\UCSD\256\philosophical_oracle\philosophy_oracle\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
filename_category_mapping, category_label_mapping = loader.load_labeling_mappings()
filename_label_mapping = filename_category_mapping
for key, value in filename_label_mapping.items():
    filename_label_mapping[key] = int(category_label_mapping[value])

loader_params = {
    "num_labels": len(category_label_mapping),
    "batch_size": 4,
    "chunk_size": 2000,
    "chunk_overlap": 100,
    "balance_multiplier": 1.5,
}

In [3]:
model_params = {
    "input_dim": 384,
    "hidden_dim": 64,
    "output_dim": loader_params['num_labels'],
    "dropout": 0.35
}
train_params = {
    "epochs": 10,
    "learning_rate": 1e-3
}

train_splits = {
    "source_texts": ["base_texts"],
    "human_summaries": ["wikipedia", "philosophize_this_transcripts"],
    "gpt_data": ["gpt_summaries", "gpt_philosophy_game"],
    "all_human_texts": ["base_texts", "wikipedia", "philosophize_this_transcripts"],
    "all_summaries": ["wikipedia", "philosophize_this_transcripts", "gpt_summaries", "gpt_philosophy_game"],
    "all_sources": ["base_texts", "wikipedia", "philosophize_this_transcripts", "gpt_summaries", "gpt_philosophy_game"],
}
eval_losses = {
    "source_texts": [],
    "human_summaries": [],
    "gpt_data": [],
    "all_human_texts": [],
    "all_summaries": [],
    "all_sources": [],
} 

test_folders = ["eval_data"]
filename_category_mapping, category_label_mapping = loader.load_labeling_mappings()

for t in range(10):
    for data_split, train_folders in train_splits.items():
        train_dataloader = loader.get_dataloader(train_folders, loader_params, filename_label_mapping, balance_data=True, print_info=False)
        test_dataloader = loader.get_dataloader(test_folders, loader_params, filename_label_mapping, balance_data=False, print_info=False)

        model = models.Classifier(**model_params)
        optim = torch.optim.Adam(model.parameters(), lr=train_params['learning_rate'])
        model.to(device)

        print(f"Training model on '{data_split}' data split")
        print(f"Epochs  || Train Loss | Train Acc || Test Loss | Test Acc")
        for epoch in range(train_params['epochs']):
            train_loss = []
            num_correct, num_total = 0, 0
            for iter, (embeddings, labels) in enumerate(train_dataloader):
                embeddings, labels = embeddings.to(device), labels.to(device)        
                optim.zero_grad()
                logits = model(embeddings)
                
                loss, correct_preds = utility.compute_loss_preds(logits, labels)
                loss.backward()
                optim.step()

                num_correct += correct_preds
                num_total += labels.shape[0]
                train_loss.append(loss.cpu().item())

            eval_loss, eval_acc = utility.evaluate_model(model, test_dataloader, device)
            train_loss = torch.tensor(train_loss)
            print(f" {epoch+1:<2}/{train_params['epochs']:>3} ||    {train_loss.mean():.3f}   |   {num_correct/num_total:.3f}   ||   {eval_loss:.3f}   |  {eval_acc:.3f}  ")
        
        eval_losses[data_split].append((eval_loss, eval_acc))

Dataloader from ('['data\\base_texts']') created with 42 batches in 10.5 seconds.
Dataloader from ('['data\\eval_data']') created with 14 batches in 1.3 seconds.
Training model on 'source_texts' data split
Epochs  || Train Loss | Train Acc || Test Loss | Test Acc
 1 / 10 ||    1.945   |   0.150   ||   1.924   |  0.283  
 2 / 10 ||    1.922   |   0.162   ||   1.913   |  0.283  
 3 / 10 ||    1.868   |   0.359   ||   1.915   |  0.208  
 4 / 10 ||    1.756   |   0.503   ||   1.919   |  0.075  
 5 / 10 ||    1.518   |   0.599   ||   1.912   |  0.132  
 6 / 10 ||    1.254   |   0.677   ||   1.952   |  0.245  
 7 / 10 ||    1.011   |   0.731   ||   2.034   |  0.226  
 8 / 10 ||    0.811   |   0.784   ||   2.131   |  0.208  
 9 / 10 ||    0.733   |   0.766   ||   2.204   |  0.189  
 10/ 10 ||    0.567   |   0.868   ||   2.243   |  0.226  
Dataloader from ('['data\\wikipedia', 'data\\philosophize_this_transcripts']') created with 65 batches in 2.2 seconds.
Dataloader from ('['data\\eval_data']